# Collect list of files with CCTP content and EBP reference

In [ ]:
import pandas as pd
import numpy as np
import re
import os 
import time

from skrub import TableReport

# import fonctions locales
import BidManagement.bid_utils as bid_utils

pd.set_option('display.width', 500)

Extraction du contenu du log de script scannant les repertoires onedrive des CSPS
- num commande EBP => construction df_EBP
- filename de CCTP, CCAP, RC,  => construction df_consult
- fichiers rejets => df_rejet

In [3]:
EBP = False
CONSULT = False
REJET = False
df2 = pd.DataFrame()
rep_data =r"C:\DocPerso\Dev_Python\Data\ATAE"
file_script_result = file_path = os.path.join(rep_data, "result_scan_AO_v5.txt")

with open(file_script_result, 'r', encoding='utf-8') as file:
    print(f"Contenu de '{file_script_result}':")
    for line_number, line in enumerate(file, 1):
        cur_line = line.strip()
        if "=== Liste ref EBP ===" in line:
            print("Processing des EBP")
            EBP = True
            continue
        elif "=== Liste fichiers CONSULT ===" in cur_line:
            #print("total EBP file:", nb_EBP_file, " Valid EBP:", nb_valid_EBP, "nb line read:",line_number)
            line_num_ebp = line_number
            print("Processing des fichiers de consultation")
            EBP = False
            CONSULT = True
            continue
        elif "=== Liste fichiers REJET ===" in cur_line:
            #print("total EBP file:", nb_EBP_file, " Valid EBP:", nb_valid_EBP, "nb line read:",line_number)
            line_num_ebp = line_number
            print("Processing des fichiers de REJET")
            EBP = False
            CONSULT = False
            REJET = True
            continue
        else:
            parts = line.split("\\jch_m\\ATAE\\") 
            #print(parts)
            if len(parts) > 1:
                line_segment = parts[1].split("\\")
                #print(line_segment)
                new_row_EBP = {0:''}
                if EBP:
                    new_row_EBP[0] = 'EBP'
                    for i in range(len(line_segment)):
                        new_row_EBP[i+1] = line_segment[i]
                    df2 = pd.concat([df2, pd.DataFrame([new_row_EBP])], ignore_index=True)
                if CONSULT:
                    new_row_EBP[0] = 'CONSULT'
                    for i in range(len(line_segment)):
                        new_row_EBP[i+1] = line_segment[i]
                    df2 = pd.concat([df2, pd.DataFrame([new_row_EBP])], ignore_index=True)
                if REJET:
                    new_row_EBP[0] = 'REJET'
                    for i in range(len(line_segment)):
                        new_row_EBP[i+1] = line_segment[i]
                    df2 = pd.concat([df2, pd.DataFrame([new_row_EBP])], ignore_index=True)

print("Shape du log scan df2",df2.shape)

if df2.shape[0] > 100_000:
    rep_data_output = r"C:\DocPerso\Dev_Python\ProjetsPerso\BidManagement\data_output"
    path_scan_df2 = os.path.join(rep_data_output, "scan_df2.pkl")

    try:
        # Save the DataFrame to a pickle file
        df2.to_pickle(path_scan_df2)
        print(f"\nDataFrame successfully saved to '{path_scan_df2}'")
    except Exception as e:
        print(f"Error saving DataFrame: {e}")


Contenu de 'C:\DocPerso\Dev_Python\Data\ATAE\result_scan_AO_v5.txt':
Processing des EBP
Processing des fichiers de consultation
Processing des fichiers de consultation


In [ ]:
##### NEW Version : EBP

df_EBP = pd.DataFrame()

df2_ebp = df2[df2[0] == "EBP"].copy() 

for i in range(len(df2_ebp)):
    ##### Extraction CSPS Name
    if isinstance(df2_ebp.iloc[i, 1], str):
        extracted_name = df2_ebp.iloc[i, 1]
    else:
        continue

    # Extraction EBP ID
    #print(df2_ebp[i][5],  df2_ebp[i][5].astype())
    if isinstance(df2_ebp.iloc[i, 5], str):
        match_numero = re.search(r"DE EBP (\d+)", df2_ebp.iloc[i, 5], re.IGNORECASE)
        numero_commande = match_numero.group(1) if match_numero else "Non trouvé"
    else:
        continue

    if numero_commande != "Non trouvé":
        # Extract EBP status
        #['04-ARCHIVES', '03-ARCHIVES', '01-CHANTIERS', '00-DEVIS', '03- ARCHIVES', '03- ARCHIVE', '03-ARCHIVE', '02-AO DEVIS A CHIFFRER', 'DOCS ATAE', 'commercial']
        directory_name = df2_ebp.iloc[i, 2]
        if "ARCHIVE" in directory_name:
            ebp_status = "Archive"
        elif "DEVIS" in directory_name:
            ebp_status = "Devis"
        elif "CHANTIER" in directory_name:
            ebp_status = "Chantier"
        else:
            continue # Passer à l'item suivant
            
        # Extraction ville Entreprise Mission
        match_details = re.search(r"([^\\]+) - ([^\\]+) - ([^\\]+)$", df2_ebp.iloc[i, 4])                
        if match_details:
            ville_code_dep = match_details.group(1)
            entreprise = match_details.group(2)
            mission_name = match_details.group(3)
            
            #print(f"Commande:{numero_commande} Ville:{ville_code_dep} Entreprise:{entreprise} Mission:{mission_name}")
            nb_valid_EBP += 1

            # Create a new row as a dictionary
            new_row_EBP = {
                'SPS Name': extracted_name.strip(),
                'ID EBP': numero_commande.strip(),
                'Ville': ville_code_dep.strip(),
                'Entreprise': entreprise.strip(),
                'Mission': mission_name.strip(),
                'statut': ebp_status,
            }
            df_EBP = pd.concat([df_EBP, pd.DataFrame([new_row_EBP])], ignore_index=True)
        else:
            #print("Détails EBP (ville, entreprise, rénovation) : Non trouvés")
            pass


print("Shape du df_EBP",df_EBP.shape)
path_df_EBP = os.path.join(rep_data_output, "df_EBP.pkl")

if df_EBP.shape[0]>2000:
    try:
        # Save the DataFrame to a pickle file
        df_EBP.to_pickle(path_df_EBP)
        print(f"\nDataFrame successfully saved to '{path_df_EBP}'")
    except Exception as e:
        print(f"Error saving DataFrame: {e}")

In [ ]:
##### NEW Version : CONSULT
##### 

df_consult = pd.DataFrame()

df2_consult = df2[df2[0] == "CONSULT"].copy() 

for i in range(len(df2_consult)):
    ##### init
    numero_commande = ""
    ebp_status =  "not defined"
    
    ##### Extraction CSPS Name
    if isinstance(df2_consult.iloc[i, 1], str):
        extracted_name = df2_consult.iloc[i, 1]
        
        # re-create full_file path 
        valid_path_components = []
        current_row_series = df2_consult.iloc[i]
        for item in current_row_series:    
            if pd.isna(item) or item is None or item=="CONSULT":
                continue # Skip NaN or None values
            
            item_str = str(item) # Convert to string
            if item_str.lower() == 'nan' or not item_str.strip(): # Also skip string 'nan' or empty strings
                continue
            
            valid_path_components.append(item_str)

        part_file_path = os.path.join(*valid_path_components)
        file_path = os.path.join(rep_data, part_file_path)
        #print(f"Generated file_path: {file_path}\n")
        #if i > 3 :
        #    break
    
        #['04-ARCHIVES', '03-ARCHIVES', '01-CHANTIERS', '00-DEVIS', '03- ARCHIVES', '03- ARCHIVE', '03-ARCHIVE', '02-AO DEVIS A CHIFFRER', 'DOCS ATAE', 'commercial']    
        # init ebp status with same logic than EBP
        directory_name = df2_consult.iloc[i, 2]
        if "ARCHIVE" in directory_name:
            ebp_status = "Archive"
        elif "DEVIS" in directory_name:
            ebp_status = "Devis"
        elif "CHANTIER" in directory_name:
            ebp_status = "Chantier"
        else:
            continue # Passer à l'item suivant
        
        # then update ebp_status with Annule, Perdu , Non Repondu if applicable
        if "annul" in file_path.lower():
            ebp_status = "Annule"
        elif "echec" in file_path.lower() or "échec" in file_path.lower():
            ebp_status = "Perdu"
        elif "non repondu" in file_path.lower() or "non répondu" in file_path.lower():
            ebp_status = "Non repondu"

        # Extraction ville Entreprise Mission
        for j in range(4, 8): 
            current_cell_value = df2_consult.iloc[i, j]
            column_name = df2_consult.columns[j] # Get the actual column name for better printing
            if pd.isna(current_cell_value) or current_cell_value is None:
                continue # Move to the next column (j+1)
            string_to_search = str(current_cell_value)
            if string_to_search.lower() == 'nan' or not string_to_search.strip():
                continue # Move to the next column (j+1)
            
            match_details = re.search(r"([^\\]+) - ([^\\]+) - ([^\\]+)$", string_to_search)
            if not match_details:
                #print(f"    Skipping: No regex match found for '{string_to_search}'.")
                continue # Move to the next column (j+1)
            else:
                # If all checks pass, extract the groups
                ville_code_dep = match_details.group(1)
                entreprise = match_details.group(2)
                mission_name = match_details.group(3)

                # Create a new row as a dictionary
                new_row_EBP = {
                    'SPS Name': extracted_name.strip(),
                    'ID EBP': numero_commande.strip(),
                    'Ville': ville_code_dep.strip(),
                    'Entreprise': entreprise.strip(),
                    'Mission': mission_name.strip(),
                    'statut': ebp_status,
                    'file_path' : file_path
                }
                df_consult = pd.concat([df_consult, pd.DataFrame([new_row_EBP])], ignore_index=True)
        else:
            #print("Détails EBP (ville, entreprise, rénovation) : Non trouvés")
            pass
    else:
        continue


print("Shape du df_consult",df_consult.shape)
path_df_consult = os.path.join(rep_data_output, "df_consult.pkl")

if df_consult.shape[0]>100_000:
    try:
        # Save the DataFrame to a pickle file
        df_consult.to_pickle(path_df_consult)
        print(f"\nDataFrame successfully saved to '{path_df_consult}'")
    except Exception as e:
        print(f"Error saving DataFrame: {e}")

In [26]:
##### NEW Version : REJET
##### 

df_rejet = pd.DataFrame()

df2_rejet = df2[df2[0] == "REJET"].copy() 

for i in range(len(df2_rejet)):
    ##### Extraction CSPS Name
    if isinstance(df2_rejet.iloc[i, 1], str):
        extracted_name = df2_rejet.iloc[i, 1]
        
        # re-create full_file path 
        valid_path_components = []
        current_row_series = df2_rejet.iloc[i]
        for item in current_row_series:    
            if pd.isna(item) or item is None or item=="REJET":
                continue # Skip NaN or None values
            
            item_str = str(item) # Convert to string
            if item_str.lower() == 'nan' or not item_str.strip(): # Also skip string 'nan' or empty strings
                continue
            
            valid_path_components.append(item_str)

        part_file_path = os.path.join(*valid_path_components)
        file_path = os.path.join(rep_data, part_file_path)
        #print(f"Generated file_path: {file_path}\n")
        #if i > 3 :
        #    break
        
        # init ebp_status et ebp_id  
        ebp_status = "Perdu"
        ebp_id = "not defined"

        # Extraction ville Entreprise Mission
        for j in range(4, 8): 
            current_cell_value = df2_rejet.iloc[i, j]
            if pd.isna(current_cell_value) or current_cell_value is None:
                continue # Move to the next column (j+1)
            string_to_search = str(current_cell_value)
            if string_to_search.lower() == 'nan' or not string_to_search.strip():
                continue # Move to the next column (j+1)
            
            match_details = re.search(r"([^\\]+) - ([^\\]+) - ([^\\]+)$", string_to_search)
            if not match_details:
                continue # Move to the next column (j+1)
            else:
                # If all checks pass, extract the groups
                ville_code_dep = match_details.group(1)
                entreprise = match_details.group(2)
                mission_name = match_details.group(3)

                # Create a new row as a dictionary
                new_row_EBP = {
                    'SPS Name': extracted_name.strip(),
                    'ID EBP': ebp_id,
                    'Ville': ville_code_dep.strip(),
                    'Entreprise': entreprise.strip(),
                    'Mission': mission_name.strip(),
                    'statut': ebp_status,
                    'file_path' : file_path
                }
                df_rejet = pd.concat([df_rejet, pd.DataFrame([new_row_EBP])], ignore_index=True)
        else:
            #print("Détails EBP (ville, entreprise, rénovation) : Non trouvés")
            pass
    else:
        continue

print("Shape du df_rejet",df_rejet.shape)
path_df_rejet = os.path.join(rep_data_output, "df_rejet.pkl")

if df_rejet.shape[0]>50:
    try:
        # Save the DataFrame to a pickle file
        df_rejet.to_pickle(path_df_rejet)
        print(f"\nDataFrame successfully saved to '{path_df_rejet}'")
    except Exception as e:
        print(f"Error saving DataFrame: {e}")

Shape du df_rejet (172, 7)

DataFrame successfully saved to 'C:\DocPerso\Dev_Python\ProjetsPerso\BidManagement\data_output\df_rejet.pkl'


In [28]:
df_consult['statut'].value_counts()

statut
Archive        106719
Chantier        26925
Perdu            4591
Devis            4401
Annule           3118
Non repondu       116
Name: count, dtype: int64

In [30]:
df_EBP["statut"].value_counts()

statut
Chantier    1578
Archive     1012
Devis          1
Name: count, dtype: int64

matched_df = pd.merge(df_EBP, df_consult, on=match_columns, how='inner', suffixes=('_ebp', '_consult'))
    there are  433  EBP ID matching
    Taille du fichier consolidé (ID EBP/Consult):1518, Taille du fichier EBP:3151, Taille du fichier consult:2027
    Nb unique EBP ID dans fichier consolidé:433, Nb unique EBP ID dans le fichier EBP:2685 

    

In [ ]:
# Merge df_consult and df_EBP pour sync EBP ID on the mission/files
# note: no need to sync EBP with rejet, because none of the Rejet file match an EBP entry (dans la liste des repertoires et fichiers). normal ?

# Define the columns to match on
match_columns = ['SPS Name', 'Ville', 'Entreprise', 'Mission']

df_consult_ebp = pd.merge(df_consult, df_EBP, on=match_columns, how='left', suffixes=('_consult', '_ebp'))
df_consult_ebp['ID EBP'] = df_consult_ebp['ID EBP_ebp'].fillna("no EBP")
df_consult_ebp = df_consult_ebp.drop(columns=['ID EBP_ebp','ID EBP_consult', 'statut_ebp'])


In [34]:
TableReport(df_consult_ebp)

Processing column   7 / 7


,,,,,,,


In [ ]:
#matched_df[[matched_df['ID EBP_ebp'] == 'Non trouvé']]
filtered_df = matched_df[matched_df['ID EBP_ebp'] == 'Non trouvé']
print(filtered_df.shape)
print(filtered_df)

filtered_df2 = filtered_df[filtered_df['filename'].isna()]
print(filtered_df2.shape)

"""
filtered_df3 = filtered_df[~filtered_df['filename'].isna()]
print(filtered_df3.shape)
print(filtered_df3)
"""
filtered_df3 = matched_df[~matched_df['filename'].isna()]
print(filtered_df3.shape)
print(filtered_df3)


In [ ]:
TableReport(filtered_df3)

In [ ]:
filtered_df4 = df_consult[df_consult['statut'] == "Offre en cours de decision"]
print(filtered_df4.shape)
print(filtered_df4)

In [ ]:
print("nb de lignes dont le statut est different : ", len(matched_df[matched_df['statut_consult'] != matched_df['statut_ebp']]))
print("nb de lignes dont le directory est different : ", len(matched_df[matched_df['Directory_consult'] != matched_df['Directory_ebp']]))
print(matched_df[matched_df['Directory_consult'] != matched_df['Directory_ebp']]['Directory_consult'][0])
print(matched_df[matched_df['Directory_consult'] != matched_df['Directory_ebp']]['Directory_ebp'][0])

In [ ]:
matched_df =  matched_df.drop(columns=['ID EBP_consult','Directory_ebp'])

In [ ]:
TableReport(matched_df)

# Test de modele pour resumer et extraire l'info un CCTP 

In [ ]:
# test load file

num_cctp_test = 0

path_root = r"C:\Users\jch_m\ATAE"
print(matched_df['SPS Name'][num_cctp_test])
print(matched_df['Directory_consult'][num_cctp_test])
print(matched_df['filename'][num_cctp_test])

file_path = os.path.join(path_root, matched_df['SPS Name'][num_cctp_test], matched_df['Directory_consult'][num_cctp_test], matched_df['filename'][num_cctp_test])
print(file_path)


In [ ]:
from pdfquery import PDFQuery
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

pdf = PDFQuery(file_path)
pdf.load()

# Use CSS-like selectors to locate the elements
text_elements = pdf.pq('LTTextLineHorizontal')

# Extract the text from the elements
text = [t.text for t in text_elements if t.text != '' or t.text != ' ']

#print(text)

In [ ]:
from ollama import Client

ollama_url = "http://localhost:11434"

# 'mistral-small3.1' #14G
list_model = ["gemma3:4b", "llama3.2", "minicpm-v", "mistral-small3.1"]
# Get an ollama client
llmclient = Client(host=ollama_url)

model_options = {
    "num_predict": 1300,  # max number of tokens to predict
    "temperature": 0.1,
    "top_p": 0.9,
}

# prompt_task = "summarize in french this text and highlight key points of the mission and bid features to consider in less than 900 words\n"
prompt_task = "En Francais, résume ce texte et extrait les points importants de la mission et les caractéristiques du contrat (prix, durée, lieu, nom du maitre d'oeuvre, nom du lieu). Donne uniquement les faits et informations disponible de text , sans interpretation ou estimation. Donne des reponses chiffrées et quantifiées lors que c'est possible. La reponse doit avoir un maximum 900 mots\n"

prompt_full = prompt_task + "".join(text)

for i in range(len(list_model)):
    model_name = list_model[i]
    print("\n==========================================")
    print(f"===== Test model :{i+1}/{len(list_model)}: {model_name} =========\n")
    time_start = time.time()
    result = llmclient.generate(
        model=model_name, prompt=prompt_full, options=model_options
    )
    print(f"\nDuree execution ={time.time()-time_start:.0f}secondes")

    pprint(result, compact=False, sort_dicts=False)
    bid_utils.print_text_wrapped("\nLLM response:\n" + result["response"])

# Etude des Rejets

In [ ]:
print(df_rejet['SPS Name'][20])
print(df_rejet['Directory'][20])
print(df_rejet['filename'][20])

file_path = os.path.join(path_root, df_rejet['SPS Name'][20], df_rejet['Directory'][20], df_rejet['filename'][20])
print(file_path)
print(os.path.exists(file_path))

In [ ]:
list_fichier_rejet_valid = []

for i in range(len(df_rejet)):
    file_path = os.path.join(path_root, df_rejet['SPS Name'][i], df_rejet['Directory'][i], df_rejet['filename'][i])
    if os.path.exists(file_path):
        try: # certains nom de fichiers provoquent des erreurs
            pdf = PDFQuery(file_path)
            pdf.load()

            # Attention les documents scannés (image) ne chargent aucun texte avec PDFQuery

            # Use CSS-like selectors to locate the elements
            text_elements = pdf.pq('LTTextLineHorizontal')
            if len(text_elements) > 20:
                #print(f"Found {len(text_elements)} LTTextLineHorizontal elements.")

                # Extract the text from the elements
                text = [t.text for t in text_elements if t.text != '' or t.text != ' ']
                list_fichier_rejet_valid.append(i)
            else:
                print(i, "=== Pas de text ", file_path )
                pass
        except Exception as e:
            print(f"*** Error processing {file_path}: {e}")        
    else:
        print(i, "=== Existe pas ", file_path )
        pass


In [ ]:
print("Nb de fichiers de rejet valides :",len(list_fichier_rejet_valid), " sur ", len(df_rejet), "fichiers")
print(list_fichier_rejet_valid)

#file_path = os.path.join(path_root, df_rejet['SPS Name'][104], df_rejet['Directory'][104], df_rejet['filename'][104])
item_valid_rejet_file = np.random.choice(list_fichier_rejet_valid, 1)[0]
file_path = os.path.join(path_root, df_rejet['SPS Name'][item_valid_rejet_file], df_rejet['Directory'][item_valid_rejet_file], df_rejet['filename'][item_valid_rejet_file])
print(file_path)

In [ ]:
# Creer une liste des documents de rejet "valide" (fichier exist et contient du text)
#
item_valid_rejet_file = np.random.choice(list_fichier_rejet_valid, 1)[0]
file_path = os.path.join(path_root, df_rejet['SPS Name'][item_valid_rejet_file], df_rejet['Directory'][item_valid_rejet_file], df_rejet['filename'][item_valid_rejet_file])
if os.path.exists(file_path):
    print("Fichier choisi :",file_path)
    pdf = PDFQuery(file_path)
    pdf.load()
    # Attention les documents scannés (image) ne chargent aucun texte aavec PDFQuery

    # Use CSS-like selectors to locate the elements
    text_elements = pdf.pq('LTTextLineHorizontal')
    if len(text_elements) > 20:
        #print(f"Found {len(text_elements)} LTTextLineHorizontal elements.")

        # Extract the text from the elements
        text = [t.text for t in text_elements if t.text != '' or t.text != ' ']
    else:
        print("ERREUR : pas de text dans le pdf")
else:
    print("ERREUR: le fichier n'est pas accessible ")


In [ ]:
# Execution du modele pour 3 exemples de fichiers "rejet" ayant du text_elements
model_name = 'llama3.2:latest'

model_options = {
    'num_predict': 1300,  # max number of tokens to predict
    'temperature': 0.1,
    'top_p': 0.9,
}

prompt_task = "summarize this text in french and highlight the reason for the ATAE bid failure, the motivation for the decision , the bid winner name and the price awarded in less than 500 words\n"

# test 3 fichiers parmi les valides
for i in range(3):
    print("\n====== Test ", i+1)
    item_valid_rejet_file = np.random.choice(list_fichier_rejet_valid, 1)[0]
    file_path = os.path.join(path_root, df_rejet['SPS Name'][item_valid_rejet_file], df_rejet['Directory'][item_valid_rejet_file], df_rejet['filename'][item_valid_rejet_file])
    if os.path.exists(file_path):
        print("Fichier choisi :", item_valid_rejet_file," : ",file_path)
        pdf = PDFQuery(file_path)
        pdf.load()
        # Attention les documents scannés (image) ne chargent aucun texte aavec PDFQuery

        # Use CSS-like selectors to locate the elements
        text_elements = pdf.pq('LTTextLineHorizontal')
        if len(text_elements) > 20:
            #print(f"Found {len(text_elements)} LTTextLineHorizontal elements.")

            # Extract the text from the elements
            text = [t.text for t in text_elements if t.text != '' or t.text != ' ']
        else:
            print("ERREUR : pas de text dans le pdf")
            break
    else:
        print("ERREUR: le fichier n'est pas accessible ")
        break
        
    prompt_full = prompt_task + "".join(text)

    result = llmclient.generate(model=model_name, prompt=prompt_full, options=model_options)

    pprint(result, compact=False, sort_dicts=False)

    #print("LLM response:\n", result['response'])
    bid_utils.print_text_wrapped("\nLLM response:\n" + result['response'])

In [ ]:
item_valid_rejet_file = 1 #55 94
file_path = os.path.join(path_root, df_rejet['SPS Name'][item_valid_rejet_file], df_rejet['Directory'][item_valid_rejet_file], df_rejet['filename'][item_valid_rejet_file])
if os.path.exists(file_path):
    print("Fichier choisi :", item_valid_rejet_file," : ",file_path)
    pdf = PDFQuery(file_path)
    pdf.load()
    # Attention les documents scannés (image) ne chargent aucun texte aavec PDFQuery

    # Use CSS-like selectors to locate the elements
    text_elements = pdf.pq('LTTextLineHorizontal')
    if len(text_elements) > 20:
        #print(f"Found {len(text_elements)} LTTextLineHorizontal elements.")

        # Extract the text from the elements
        text = [t.text for t in text_elements if t.text != '' or t.text != ' ']
    else:
        print("ERREUR : pas de text dans le pdf")
        
else:
    print("ERREUR: le fichier n'est pas accessible ")
    
    
prompt_full = prompt_task + "".join(text)

result = llmclient.generate(model=model_name, prompt=prompt_full, options=model_options)

pprint(result, compact=False, sort_dicts=False)

print("LLM response:\n", result['response'])

## Consolidation rejet

In [ ]:
TableReport(df_rejet)

In [ ]:
# Define the columns to match on
match_columns = ['SPS Name', 'Ville', 'Entreprise', 'Mission']

# Perform an inner merge on the specified columns
# An inner merge will only keep rows where the values in 'match_columns' are identical in both DataFrames.
reject_c_df = pd.merge(df_rejet, df_consult, on=match_columns, how='inner', suffixes=('_rej', '_consult'))

print("there are ",len(reject_c_df), " rejets matching")
print(f"Taille du fichier consolidé (consult/rejet):{len(reject_c_df)}, Taille du fichier consult:{len(df_consult)}, Taille du fichier rejet:{len(df_rejet)}")
#print(f"Nb unique EBP ID dans fichier consolidé:{len(reject_c_df['ID EBP_ebp'].unique())}, Nb unique EBP ID dans le fichier EBP:{len(df_EBP['ID EBP'].unique())} ")

print(reject_c_df.head())

In [ ]:
filtered_consult2 = df_consult[ (df_consult['Ville'] == "PORNICHET") & (df_consult['Entreprise'] == "MAIRIE") ].head(10)
print(filtered_consult2.head(10))

In [ ]:
TableReport(df_consult)